In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models import ChatOpenAI


d:\Pritesh\VS Code Workspace\Preparations\preparations\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
@dataclass
class TableSchema:
    name: str
    description: str
    columns: Dict[str, str]  # column_name: description #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
    relationships: Dict[str, str]  # column_name: related_table.column #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
    key_metrics: List[str]
    sample_queries: List[str]

class UnderwritingRAG:
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.embedding_model = SentenceTransformer(model_name)
        self.index = None
        self.schema_store = {}
        self.dataframes = {}
        
        self._initialize_schemas() #--------------------------------------------->>>>>>>>>>>>>>>>>>>>> Initialize the schema

    def _initialize_schemas(self):
        """Initialize schema metadata for all tables"""
        self.schema_store = {
            "risk_assessment": TableSchema(
                name="Risk Assessment",
                description="Contains risk assessment details and recommendations for borrowers",
                columns={
                    "assessment_id": "Unique identifier for risk assessment",
                    "borrower_id": "Foreign key to Borrower Profile",
                    "risk_score": "Overall risk score (0-100)",
                    "market_risk": "Market risk level (Low/Moderate/High)",
                    "credit_risk": "Credit risk level (Low/Moderate/High)",
                    "operational_risk": "Operational risk level (Low/Moderate/High)",
                    "regulatory_risk": "Regulatory risk level (Low/Moderate/High)",
                    "recommendation": "Final recommendation (Approve/Reject/Further Review)",
                    "assessment_date": "Date of risk assessment",
                    "reviewed_by": "Name of reviewing officer",
                    "next_review_date": "Next scheduled review date"
                },
                relationships={
                    "borrower_id": "borrower_profile.borrower_id"
                },
                key_metrics=[
                    "risk_score",
                    "recommendation",
                    "risk_levels"
                ],
                sample_queries=[
                    "What is the risk assessment for borrower TCS-1?",
                    "Show all high credit risk assessments",
                    "List rejected applications with risk scores above 50"
                ]
            ),
            
            "borrower_profile": TableSchema(
                name="Borrower Profile",
                description="Contains company profile and credit rating information",
                columns={
                    "borrower_id": "Unique identifier for borrower",
                    "company_name": "Name of the company",
                    "industry_sector": "Industry sector of the company",
                    "incorporation_date": "Date of incorporation",
                    "headquarters_location": "Company headquarters location",
                    "business_structure": "Legal structure of business",
                    "ownership": "Ownership details",
                    "credit_rating": "Current credit rating",
                    "last_rating_update": "Last credit rating update date"
                },
                relationships={},
                key_metrics=[
                    "credit_rating",
                    "business_age",
                    "rating_trend"
                ],
                sample_queries=[
                    "What is TCS-1's credit rating?",
                    "List all AAA rated borrowers",
                    "Show recently incorporated companies"
                ]
            ),
            
            "credit_history": TableSchema(
                name="Credit History",
                description="Loan and credit history details",
                columns={
                    "credit_id": "Unique identifier for credit record",
                    "borrower_id": "Foreign key to Borrower Profile",
                    "credit_type": "Type of credit facility",
                    "loan_amount": "Original loan amount",
                    "loan_issued_date": "Date loan was issued",
                    "loan_maturity_date": "Loan maturity date",
                    "repayment_status": "Current repayment status",
                    "default_history": "Previous default history",
                    "interest_rate": "Current interest rate",
                    "outstanding_balance": "Current outstanding balance"
                },
                relationships={
                    "borrower_id": "borrower_profile.borrower_id"
                },
                key_metrics=[
                    "default_rate",
                    "average_loan_amount",
                    "repayment_performance"
                ],
                sample_queries=[
                    "Show default history for borrower TCS-1",
                    "List all active loans above $40000",
                    "What is the average interest rate for term loans?"
                ]
            ),
            
            "financial_ratios": TableSchema(
                name="Financial Ratios",
                description="Key financial ratios and performance metrics",
                columns={
                    "ratio_id": "Unique identifier for ratio record",
                    "borrower_id": "Foreign key to Borrower Profile",
                    "reporting_period": "Period of financial reporting",
                    "current_ratio": "Current assets / Current liabilities",
                    "quick_ratio": "Quick assets / Current liabilities",
                    "debt_to_equity": "Total debt / Total equity",
                    "interest_coverage_ratio": "EBIT / Interest expenses",
                    "gross_margin_ratio": "Gross profit / Revenue",
                    "return_on_assets": "Net income / Total assets",
                    "return_on_equity": "Net income / Shareholder equity"
                },
                relationships={
                    "borrower_id": "borrower_profile.borrower_id"
                },
                key_metrics=[
                    "solvency_metrics",
                    "profitability_metrics",
                    "efficiency_metrics"
                ],
                sample_queries=[
                    "What are the current ratios below 1.5?",
                    "Show companies with high debt-to-equity",
                    "List top 10 by return on equity"
                ]
            ),
            
            "financial_statements": TableSchema(
                name="Financial Statements",
                description="Detailed financial statement data",
                columns={
                    "statement_id": "Unique identifier for financial statement",
                    "borrower_id": "Foreign key to Borrower Profile",
                    "reporting_period": "Period of financial reporting",
                    "total_assets": "Total assets value",
                    "total_liabilities": "Total liabilities value",
                    "shareholder_equity": "Total shareholder equity",
                    "total_revenue": "Total revenue",
                    "net_income": "Net income",
                    "cash_flow_operations": "Cash flow from operations",
                    "cash_flow_investments": "Cash flow from investments",
                    "cash_flow_financing": "Cash flow from financing",
                    "gross_margin": "Gross margin percentage",
                    "operating_margin": "Operating margin percentage",
                    "net_profit_margin": "Net profit margin percentage"
                },
                relationships={
                    "borrower_id": "borrower_profile.borrower_id"
                },
                key_metrics=[
                    "size_metrics",
                    "profitability_metrics",
                    "cash_flow_metrics"
                ],
                sample_queries=[
                    "Show companies with negative operating cash flow",
                    "List top 10 by revenue",
                    "What is the average profit margin?"
                ]
            )
        }

    def create_schema_embedding(self, table_name: str) -> str:
        """Create schema description for embedding"""
        schema = self.schema_store[table_name]
        
        schema_text = f"Table: {schema.name}\n"
        schema_text += f"Description: {schema.description}\n\n"
        
        # Add columns #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        schema_text += "Columns:\n"
        for col, desc in schema.columns.items():
            schema_text += f"- {col}: {desc}\n"
        
        # Add relationships #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        if schema.relationships:
            schema_text += "\nRelationships:\n"
            for col, rel in schema.relationships.items():
                schema_text += f"- {col} relates to {rel}\n"
        
        # Add key metrics #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        schema_text += "\nKey Metrics:\n"
        for metric in schema.key_metrics:
            schema_text += f"- {metric}\n"
        
        # Add sample queries #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        schema_text += "\nTypical Queries:\n"
        for query in schema.sample_queries:
            schema_text += f"- {query}\n"
            
        return schema_text

    def add_dataframe(self, df: pd.DataFrame, table_name: str) -> None:
        """Add a dataframe to the system"""
        self.dataframes[table_name] = df
        
        if self.index is None:
            # Create initial index with schema embedding #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
            schema_text = self.create_schema_embedding(table_name)
            embedding = self.embedding_model.encode([schema_text])[0]
            self.index = faiss.IndexFlatL2(embedding.shape[0])
            self.index.add(np.array([embedding]).astype('float32'))
        else:
            # Add schema embedding to existing index #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
            schema_text = self.create_schema_embedding(table_name)
            embedding = self.embedding_model.encode([schema_text])[0]
            self.index.add(np.array([embedding]).astype('float32'))

    def query(self, query: str, k: int = 2) -> List[Dict]:
        """Query the system and return relevant tables and data"""
        # Create query embedding #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        query_vector = self.embedding_model.encode([query])
        query_vector = np.array(query_vector).astype('float32')
        
        # Search index #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        distances, indices = self.index.search(query_vector, k)
        
        # Get relevant tables #--------------------------------------------->>>>>>>>>>>>>>>>>>>>>
        relevant_tables = []
        for idx, distance in enumerate(distances[0]):
            table_name = list(self.schema_store.keys())[indices[0][idx]]
            schema = self.schema_store[table_name]
            
            relevant_tables.append({
                'table_name': schema.name,
                'relevance_score': float(distance),
                'schema': schema,
                'sample_data': self.dataframes[table_name].head(3) if table_name in self.dataframes else None
            })
            
        return relevant_tables

def format_query_results(results: List[Dict]) -> str:
    """Format query results for LLM consumption"""
    context = "Relevant tables for your query:\n\n"
    
    for result in results:
        context += f"Table: {result['table_name']}\n"
        context += f"Relevance Score: {result['relevance_score']:.4f}\n"
        schema = result['schema']
        
        context += "Key Columns:\n"
        for col, desc in schema.columns.items():
            context += f"- {col}: {desc}\n"
        
        if result['sample_data'] is not None:
            context += "\nSample Data:\n"
            context += result['sample_data'].to_string()
        
        context += "\n\n"
    
    return context


In [ ]:

def initialize_rag_system(risk_df, borrower_df, credit_df, ratios_df, statements_df):
    rag = UnderwritingRAG()
    
    #--------------------------------------------->>>>>>>>>>>>>>>>>>>>> Add all dataframes
    rag.add_dataframe(risk_df, "risk_assessment")
    rag.add_dataframe(borrower_df, "borrower_profile")
    rag.add_dataframe(credit_df, "credit_history")
    rag.add_dataframe(ratios_df, "financial_ratios")
    rag.add_dataframe(statements_df, "financial_statements")
    
    return rag

In [8]:
csv_path = "C:/Users/prite/Downloads/Sample Underwriter/tabular/"

In [ ]:
#--------------------------------------------->>>>>>>>>>>>>>>>>>>>> Load CSV files 
risk_df = pd.read_csv(csv_path +"Risk_Assessment.csv")
borrower_df = pd.read_csv(csv_path +"Borrower_Profile.csv")
credit_df = pd.read_csv(csv_path +"Credit_History.csv")
ratios_df = pd.read_csv(csv_path +"Financial_Ratios.csv")
statements_df = pd.read_csv(csv_path +"Financial_Statements.csv")

In [23]:
model = ChatOpenAI()

In [ ]:
#--------------------------------------------->>>>>>>>>>>>>>>>>>>>> Initialize RAG system 
rag = initialize_rag_system(risk_df, borrower_df, credit_df, ratios_df, statements_df)

d:\Pritesh\VS Code Workspace\Preparations\preparations\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prite\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
#--------------------------------------------->>>>>>>>>>>>>>>>>>>>> LLM CALL
def retrieval_llm(query):

    results = rag.query(query)
    context = format_query_results(results)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system","""Based on the following underwriting data context {context}, answer the query: """),
        ("human","{query}")
        ])


    chain = prompt | model | StrOutputParser()

    final_result = chain.invoke({
        "context":context,
        "query":query
    })

    return results, context, final_result

### Questions

#### Question 1

In [28]:
query = "Fetch basic borrower profile details for TCS"
results, context, final_result = retrieval_llm(query)
print(final_result)

To fetch basic borrower profile details for TCS (Tata Consultancy Services), you can use the following SQL query:

```sql
SELECT * 
FROM `Borrower Profile`
WHERE company_name = 'Tata Consultancy Services';
```

This query will retrieve all the basic borrower profile details for TCS from the "Borrower Profile" table.


#### Question 2

In [29]:
query = "Retrieve recent financial statements for TCS (top 5 by most recent reporting period)."
results, context, final_result = retrieval_llm(query)
print(final_result)

To retrieve the recent financial statements for Tata Consultancy Services (TCS), we can query the Financial Statements table based on the borrower_id for TCS and then sort the data by the reporting_period to get the top 5 most recent financial statements. Here is the SQL query to achieve this:

```sql
SELECT *
FROM Financial_Statements FS
JOIN Borrower_Profile BP ON FS.borrower_id = BP.borrower_id
WHERE BP.company_name = 'Tata Consultancy Services'
ORDER BY FS.reporting_period DESC
LIMIT 5;
```

This query will retrieve the top 5 most recent financial statements for Tata Consultancy Services by joining the Financial Statements table with the Borrower Profile table on the borrower_id, filtering for TCS as the company name, and sorting the data by reporting_period in descending order to get the most recent statements first.


#### Question 3

In [30]:
query = "Calculate average financial ratios for TCS over the past 3 years"
results, context, final_result = retrieval_llm(query)
print(final_result)

To calculate the average financial ratios for TCS over the past 3 years, we need to consider the data from both the Financial Statements and Financial Ratios tables for TCS (borrower_id) for the relevant reporting periods.

Here is the breakdown of the steps to calculate the average financial ratios for TCS over the past 3 years:

1. Identify the financial statements and financial ratios data for TCS (borrower_id) for the past 3 years.
2. Calculate the average of each financial ratio for TCS over the past 3 years.

Let's proceed with the calculations:

1. Financial Statements Data for TCS over the past 3 years:
   - TCS-2:
     - FS-1: 2021-09-28
     - FR-1: 2023-11-29
   - TCS-3:
     - FS-2: 2024-09-22
     - FR-2: 2021-04-01
   - TCS-4:
     - FS-3: 2022-02-08
     - FR-3: 2020-11-28

2. Calculate the average financial ratios for TCS over the past 3 years:
   - Current Ratio:
     - (2.03 + 1.14 + 2.82) / 3 = 2.00
   - Quick Ratio:
     - (0.78 + 1.85 + 1.61) / 3 = 1.41
   - Debt t

#### Question 4

In [31]:
query = "List all credit transactions for TCS with status as 'Default'"
results, context, final_result = retrieval_llm(query)
print(final_result)

```sql
SELECT *
FROM Credit History
WHERE borrower_id LIKE 'TCS-%' AND repayment_status = 'Default';
```


#### Question 5

In [32]:
query = "Retrieve the most recent risk assessment details for TCS"
results, context, final_result = retrieval_llm(query)
print(final_result)

To retrieve the most recent risk assessment details for TCS (Tata Consultancy Services), you can use the following SQL query:

```sql
SELECT ra.assessment_id, ra.borrower_id, ra.assessment_date, ra.risk_score, ra.market_risk, ra.credit_risk, ra.operational_risk, ra.regulatory_risk, ra.recommendation, ra.reviewed_by, ra.next_review_date
FROM Risk_Assessment ra
JOIN Borrower_Profile bp ON ra.borrower_id = bp.borrower_id
WHERE bp.company_name = 'Tata Consultancy Services'
ORDER BY ra.assessment_date DESC
LIMIT 1;
```

This query will fetch the assessment details for TCS by joining the Risk Assessment and Borrower Profile tables on the borrower_id, filtering for TCS as the company name, ordering the results by assessment date in descending order (most recent first), and limiting the output to only the most recent assessment record.
